In [1]:
import pandas as pd
import os
import tensorflow as tf
import os
# os.chdir('D:/daewoo')
import sys
# sys.path.append('D:/daewoo')

from daewoo_module import *
import sklearn.metrics as skm
import time
from collections import Counter
import numpy as np
import tflearn

###### 데이터 불러오기, 학습하기
os.environ['FOR_DISABLE_CONSOLE_CTRL_HANDLER'] = '1'

root_dir = ".\\input_data"
img_dir = "./input_data/figure/"
logs_path = os.path.join(root_dir, "graph")

df = pd.read_csv(os.path.join(root_dir, 'description.csv'), engine='python')
    

C:\Users\Jay\Anaconda3\envs\tensorflow\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


curses is not supported on this machine (please install/reinstall curses for an optimal experience)


In [2]:
classification = False
crop = False  # True = 10 crop, False = original size
RGB = True  # True = RGB, False = Grayscale
resize = False  # True = 240x70, False = 480x135

# if crop == True:
#     batch_size = 64
# else:
#     batch_size = 4

batch_size = 16

epochs = 5


df['classification'] = pd.cut(df['WVHT ft.y'], bins=[0, 5.2, 7.9, 100], labels=[0, 1, 2], include_lowest=True).values

if classification is True:
    img = df.img_name.values
    img = np.array([img_dir + x for x in img])    
    label = pd.cut(df['WVHT ft.y'], bins=[0, 5.2, 7.9, 100], labels=[0, 1, 2], include_lowest=True).values
    idx = [i for i in range(len(label)) if label[i] != 1]
    img = img[idx]
    label = label[idx]
    for i in range(len(label)):
        if label[i] == 2:
            label[i] = 1
else:
    df = df[df['classification'] == 0]    
    img = df['img_name'].values
    img = np.array([img_dir + x for x in img])    
    _label = df['WVHT ft.y'].values
    label = ((label - np.mean(_label)) / np.std(_label)).reshape(-1, 1)


In [3]:
# Tensorflow Dataset API
train_img_tensor, train_label_tensor, test_img_tensor, test_label_tensor, tr_idx, ts_idx = set_input(img, label)

train_imgs = tf.data.Dataset.from_tensor_slices((train_img_tensor, train_label_tensor))
test_imgs = tf.data.Dataset.from_tensor_slices((test_img_tensor, test_label_tensor))
infer_imgs = tf.data.Dataset.from_tensor_slices((test_img_tensor, test_label_tensor))

# train_imgs = train_imgs.map(input_tensor_resizeX_RGB).shuffle(buffer_size=100).apply(lambda x: make_batch(x)).batch(batch_size).repeat()
# test_imgs = test_imgs.map(input_tensor_resizeX_RGB).shuffle(buffer_size=100).apply(lambda x: make_batch(x)).batch(batch_size).repeat()
train_imgs = train_imgs.map(input_tensor_resizeX_RGB).shuffle(buffer_size=100).batch(batch_size).repeat()
test_imgs = test_imgs.map(input_tensor_resizeX_RGB).shuffle(buffer_size=100).batch(batch_size).repeat()
infer_imgs = infer_imgs.map(input_tensor_resizeX_RGB).batch(batch_size)

In [4]:
train_iterator = train_imgs.make_initializable_iterator()
test_iterator = test_imgs.make_initializable_iterator()
infer_iterator = infer_imgs.make_initializable_iterator()
handle = tf.placeholder(tf.string, shape=[])

iterator = tf.data.Iterator.from_string_handle(handle, train_imgs.output_types, train_imgs.output_shapes)
x, y = iterator.get_next()

# train class: [22789, 19659]
# if crop == True:
#     train_batches = 22789 * 2 * 10 // batch_size
# else:
#     train_batches = 22789 * 2 // batch_size

train_batches = 18231 // batch_size


In [5]:

######################
##### 4개 모델 중 선택 ###
######################

model = VGG16(x, y, bn=True, classification=classification)
model_name = "VGG16_regression_low_wave_pretrained"

start_time = time.time()

config = tf.ConfigProto()
config.gpu_options.allow_growth = True

sess = tf.Session(config=config)
saver = tf.train.Saver()
sess.run(tf.global_variables_initializer())
train_handle = sess.run(train_iterator.string_handle())
test_handle = sess.run(test_iterator.string_handle())
infer_handle = sess.run(infer_iterator.string_handle())
train_writer = tf.summary.FileWriter(os.path.join(logs_path, model_name, 'train'), sess.graph)
test_writer = tf.summary.FileWriter(os.path.join(logs_path, model_name, 'test'))

LEARNING_RATE = 0.001
optimizer = model.rms


In [6]:
print("Training!")
for i in range(epochs):
    print("-------{} Epoch--------".format(i + 1))
    sess.run(train_iterator.initializer)
    sess.run(test_iterator.initializer)
    for j in range(train_batches):
        summary, _, loss_, __ = sess.run([model.merged_summary_op, optimizer, model.loss, model.logits],
                                     feed_dict={handle: train_handle, model.learning_rate: LEARNING_RATE})
        step = tf.train.global_step(sess, model.global_step)
        #print("Training Iter : {}, Loss : {:.4f}".format(step, loss_))

        if j % 10 == 0:
            train_writer.add_summary(summary, step)
            summary, loss_, _ = sess.run([model.merged_summary_op, model.loss, model.logits],
                                      feed_dict={handle: test_handle})
            print("Validation Iter : {}, Loss : {:.4f}".format(step, loss_))
            test_writer.add_summary(summary, step)

print("-----------End of training-------------")

end_time = time.time() - start_time
print("{} seconds".format(end_time))

saver.save(sess, os.path.join(logs_path, model_name))


Training!
-------1 Epoch--------
Validation Iter : 1, Loss : 16.6977
Validation Iter : 11, Loss : 15.4962
Validation Iter : 21, Loss : 5.4109
Validation Iter : 31, Loss : 1.7409
Validation Iter : 41, Loss : 3.5635
Validation Iter : 51, Loss : 2.6075
Validation Iter : 61, Loss : 1.4306
Validation Iter : 71, Loss : 1.0172
Validation Iter : 81, Loss : 3.3307
Validation Iter : 91, Loss : 2.0605
Validation Iter : 101, Loss : 2.4860
Validation Iter : 111, Loss : 3.3578
Validation Iter : 121, Loss : 0.8821
Validation Iter : 131, Loss : 2.7405
Validation Iter : 141, Loss : 10.7085
Validation Iter : 151, Loss : 3.2195
Validation Iter : 161, Loss : 0.7274
Validation Iter : 171, Loss : 4.4478
Validation Iter : 181, Loss : 2.1278
Validation Iter : 191, Loss : 8.1475
Validation Iter : 201, Loss : 1.9505
Validation Iter : 211, Loss : 2.0990
Validation Iter : 221, Loss : 2.7986
Validation Iter : 231, Loss : 2.8095
Validation Iter : 241, Loss : 1.0407
Validation Iter : 251, Loss : 4.3338
Validation It

Validation Iter : 2170, Loss : 0.6695
Validation Iter : 2180, Loss : 1.0914
Validation Iter : 2190, Loss : 2.1786
Validation Iter : 2200, Loss : 0.8667
Validation Iter : 2210, Loss : 2.1101
Validation Iter : 2220, Loss : 1.1531
Validation Iter : 2230, Loss : 1.6236
Validation Iter : 2240, Loss : 1.1716
Validation Iter : 2250, Loss : 0.7863
Validation Iter : 2260, Loss : 0.7153
Validation Iter : 2270, Loss : 1.4618
-------3 Epoch--------
Validation Iter : 2279, Loss : 1.6139
Validation Iter : 2289, Loss : 1.1868
Validation Iter : 2299, Loss : 1.1659
Validation Iter : 2309, Loss : 1.2949
Validation Iter : 2319, Loss : 1.1242
Validation Iter : 2329, Loss : 1.4471
Validation Iter : 2339, Loss : 0.9603
Validation Iter : 2349, Loss : 1.3611
Validation Iter : 2359, Loss : 1.2725
Validation Iter : 2369, Loss : 1.2989
Validation Iter : 2379, Loss : 0.8673
Validation Iter : 2389, Loss : 0.5184
Validation Iter : 2399, Loss : 1.1304
Validation Iter : 2409, Loss : 0.5924
Validation Iter : 2419, Los

Validation Iter : 4318, Loss : 0.9925
Validation Iter : 4328, Loss : 1.8276
Validation Iter : 4338, Loss : 0.7426
Validation Iter : 4348, Loss : 1.0113
Validation Iter : 4358, Loss : 1.1059
Validation Iter : 4368, Loss : 0.9069
Validation Iter : 4378, Loss : 1.5362
Validation Iter : 4388, Loss : 0.5784
Validation Iter : 4398, Loss : 1.0399
Validation Iter : 4408, Loss : 2.2243
Validation Iter : 4418, Loss : 1.2858
Validation Iter : 4428, Loss : 1.7532
Validation Iter : 4438, Loss : 0.9027
Validation Iter : 4448, Loss : 0.5616
Validation Iter : 4458, Loss : 1.3132
Validation Iter : 4468, Loss : 0.9653
Validation Iter : 4478, Loss : 1.2063
Validation Iter : 4488, Loss : 1.4417
Validation Iter : 4498, Loss : 0.8510
Validation Iter : 4508, Loss : 1.4615
Validation Iter : 4518, Loss : 2.3532
Validation Iter : 4528, Loss : 1.1493
Validation Iter : 4538, Loss : 1.6375
Validation Iter : 4548, Loss : 0.6122
-------5 Epoch--------
Validation Iter : 4557, Loss : 1.5640
Validation Iter : 4567, Los

'.\\input_data\\graph\\VGG16_regression_low_wave_pretrained'

In [ ]:
# 모델 불러오기
#os.path.join(logs_path, model_name)
# model_name = "VGG16_regression_low_wave_pretrained"
# sess = tf.Session()
# new_saver = tf.train.import_meta_graph(os.path.join(logs_path, model_name))
#new_saver.restore(sess, tf.train.latest_checkpoint(os.path.join(logs_path, model_name)))

In [7]:

####################
# Inference##########
####################

sess.run(infer_iterator.initializer)
infer_regression = np.empty((0,2))

while True:
    try:
        tmp_true, tmp_pred = sess.run([model.y, model.logits], feed_dict={handle: infer_handle})
        tmp_concat = np.concatenate((tmp_true, tmp_pred), axis=1)
        infer_regression = np.concatenate((infer_regression, tmp_concat), axis=0)
        if i % 200 == 0:
            print(i)
        i += 1
    except:
        #y_true = np.array([np.where(r == 1)[0][0] for r in y_true])
        break


200


In [38]:
mapping = lambda x : x * np.std(_label) + np.mean(_label)
a = np.array([1,2,3,4])
y_true = mapping(infer_regression[:,0])
y_true = np.round_(y_true, 2).reshape(-1,1)
y_pred = mapping(infer_regression[:,1])
y_pred = np.round_(y_pred, 2).reshape(-1,1)

infer_regression = np.concatenate((y_true, y_pred), axis=1)


np.savetxt("infer.csv", infer_regression, delimiter=",")
# infer_regression = np.genfromtxt('infer.csv', delimiter=',')
# infer_regression = pd.read_csv('infer.csv')